#  Objective - This code is built for prediction of the returning shipment

#  Description - The code will have basic 4 sections :
##                         1. Data Import and basic Quality Checks
##                         2. Exploratoy data analysis, adding features to data and making model ready data
##                         3. Building Random Forrest and Neural Network Models   
##                         4. Prediction of return rate using the validation data


# Date -            07 July 2021
# Author -        Avinash Mishra 
# Company -    XYZ

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import ta
import warnings
import time 
from IPython.display import clear_output
#from pyti.exponential_moving_average import exponential_moving_average as EMA
import xlrd 
import xlsxwriter
import openpyxl
import sys
import decimal
import pyodbc 
import pandas.io.sql
import smtplib
import mimetypes
import time
import datetime
import os
#import paho.mqtt.client as mqtt
import sys  
import csv
import socket
from datetime import datetime
from scipy.signal import argrelextrema
from scipy.signal import find_peaks, peak_prominences
import pickle
import matplotlib
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
from matplotlib import style
from functools import reduce 
import re
import seaborn as sns
import sklearn
from sklearn import preprocessing
from scipy.spatial.distance import euclidean
import glob
from datetime import date, timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from IPython.display import clear_output
from numpy import arange
import os 
from sklearn.metrics import r2_score
import ta
from ta import *
import talib
#from rfpimp import permutation_importances

### Section  - 1 - Basic Data Quality Checks

In [34]:
path = "C:/Users/Avinash Mishra/Avinash/Python Codes/Teleperformance/"
df= pd.read_csv(path + "TrainingData_V1.csv")


df.head()

,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return
0,1,22-06-2016,27-06-2016,643,38,navy,30,49.9,30822,Mrs,17-04-1969,1013,23-06-2016,0
1,10,22-06-2016,27-06-2016,195,xxl,grey,46,19.9,30823,Mrs,22-04-1970,1001,15-03-2015,1
2,11,22-06-2016,05-07-2016,25,xxl,grey,5,79.9,30823,Mrs,22-04-1970,1001,15-03-2015,0
3,32,23-06-2016,26-06-2016,173,m,brown,20,19.9,17234,Mrs,09-01-1960,1013,17-02-2015,0
4,43,23-06-2016,26-06-2016,394,40,black,44,90.0,30827,Mrs,NaN,1006,09-02-2016,1


In [35]:
#### create some derived varibale in the data 
df['delivery_date']=pd.to_datetime(df['delivery_date'],format = '%d-%m-%Y')
df['order_date']=pd.to_datetime(df['order_date'],format = '%d-%m-%Y')
df['user_reg_date']=pd.to_datetime(df['user_reg_date'],format = '%d-%m-%Y')
df['user_dob']=pd.to_datetime(df['user_dob'],format = '%d-%m-%Y')
df['Today'] =pd.to_datetime(date.today(),format = '%Y-%m-%d')


df['Delivery_Time'] = (df['delivery_date'] -df['order_date'])// timedelta(days=1)
df['User_Age_years'] = (df['Today'] - df['user_dob'])// timedelta(days=365.2425)
df['System_Age_months'] =(df['Today'] - df['user_reg_date'])// timedelta(days=30)



In [36]:
df['User_Age_years'].describe()

count    72956.000000
mean        56.729947
std         11.259442
min          9.000000
25%         51.000000
50%         56.000000
75%         62.000000
max        120.000000
Name: User_Age_years, dtype: float64

In [37]:
df['Delivery_Time'] = np.where(df['Delivery_Time']< 0, df['Delivery_Time'].mean(),df['Delivery_Time'] )
df['Delivery_Time'] = df['Delivery_Time'].fillna(df['Delivery_Time'].mean())
df['Delivery_Time'].describe()

count    79945.000000
mean         6.306488
std         15.813433
min        -84.607387
25%          2.000000
50%          3.000000
75%          6.306488
max        173.000000
Name: Delivery_Time, dtype: float64

In [38]:
### get the columns with na values 
df[df.columns[df.isnull().any()]].isnull().sum() * 100 / df.shape[0]


delivery_date     9.301395
user_dob          8.742260
User_Age_years    8.742260
dtype: float64

# Section  2 : Exploratory Analysis to understand the data 

In [39]:
df =df.sort_values(by=['order_date'], ascending=True)
df= df.reset_index()
df

,index,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return,Today,Delivery_Time,User_Age_years,System_Age_months
0,0,1,2016-06-22,2016-06-27,643,38,navy,30,49.90,30822,Mrs,1969-04-17,1013,2016-06-23,0,2021-07-06,5.000000,52.0,61
1,60177,650,2016-06-22,2016-06-28,106,38,blue,6,39.90,30931,Mr,1970-03-29,1010,2015-02-17,0,2021-07-06,6.000000,51.0,77
2,9976,2,2016-06-22,NaT,337,152,grey,30,19.95,30822,Mrs,1969-04-17,1013,2016-06-23,0,2021-07-06,6.306488,52.0,61
3,40080,574,2016-06-22,2016-07-18,315,39,grey,45,89.90,9214,Mrs,1959-02-23,1015,2015-05-13,1,2021-07-06,26.000000,62.0,74
4,40079,563,2016-06-22,2016-06-26,423,xl,red,33,12.90,30920,Mrs,1952-05-24,1002,2015-02-17,1,2021-07-06,4.000000,69.0,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79940,50076,99823,2016-09-11,2016-09-13,2048,xxl,olive,75,249.90,48213,Mr,1944-08-10,1015,2015-02-17,1,2021-07-06,2.000000,76.0,77
79941,50075,99821,2016-09-11,2016-09-13,2127,unsized,black,48,19.95,48213,Mr,1944-08-10,1015,2015-02-17,1,2021-07-06,2.000000,76.0,77
79942,50074,99806,2016-09-11,2016-09-12,292,6+,grey,4,79.90,48209,Mrs,1970-01-31,1013,2016-09-12,1,2021-07-06,1.000000,51.0,58
79943,70072,99999,2016-09-11,1994-12-31,1832,xxl,black,37,26.90,47794,Mr,1985-03-01,1006,2016-09-10,0,2021-07-06,-84.607387,36.0,58


###### Adding some derived variables in the data

#### Creating a variable for the user return history

In [40]:



#df['User_level_return_count'] = df.groupby(by=['user_id','order_date'])['return'].sum()

#### Creating historical return history at a user level 

test = pd.DataFrame(df.groupby(by=['user_id','order_date'])['return'].sum())
test =test.reset_index()
test.columns = ['user_id','order_date','return']
test['user_return_history'] = test.groupby(['user_id'])['return'].transform(pd.Series.cumsum)
test['user_return_history'] =test.groupby('user_id')['user_return_history'].shift().fillna(0)
test=test[['order_date','user_id','user_return_history']]

df = df.merge(test, how='inner', on=['order_date','user_id'])


## Recent historical return rates by item,brand,size,color and state

In [42]:
####### Recent historical return at an item and brand level 

#### list of variable to create return trend 

var = ['brand_id','item_id','item_size','user_state','item_color']

for each in var:


    test = pd.DataFrame()
    test =df.groupby(by=[each,'order_date']).agg(Total_Returns=('return', 'sum'),Total_Orders=('return', 'count'))
    test['Return_per'] = test['Total_Returns']/test['Total_Orders']
    test[str(each) + "_return_trend"] = test['Return_per'].rolling(3).mean()
    test[str(each) + "_return_trend"] =test.groupby(by =[each])[str(each) + "_return_trend"].shift().fillna(0)

    test = test.reset_index()
    test = test[['order_date',each,str(each) + "_return_trend"]]

    df = df.merge(test, how='inner', on=['order_date',each])


In [43]:
df

,index,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,...,Today,Delivery_Time,User_Age_years,System_Age_months,user_return_history,brand_id_return_trend,item_id_return_trend,item_size_return_trend,user_state_return_trend,item_color_return_trend
0,0,1,2016-06-22,2016-06-27,643,38,navy,30,49.90,30822,...,2021-07-06,5.000000,52.0,61,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,9976,2,2016-06-22,NaT,337,152,grey,30,19.95,30822,...,2021-07-06,6.306488,52.0,61,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,19997,605,2016-06-22,2016-06-27,425,xxl,grey,46,24.90,30928,...,2021-07-06,5.000000,75.0,72,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,30063,604,2016-06-22,2016-06-27,358,xxl,grey,74,39.90,30928,...,2021-07-06,5.000000,75.0,72,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,2,11,2016-06-22,2016-07-05,25,xxl,grey,5,79.90,30823,...,2021-07-06,13.000000,51.0,76,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79940,19943,99971,2016-09-11,2016-09-13,324,40,ocher,44,85.00,48237,...,2021-07-06,2.000000,55.0,77,0.0,1.000000,1.000000,0.527277,0.355311,0.409524
79941,60106,99928,2016-09-11,1994-12-31,71,9+,ocher,21,49.95,45237,...,2021-07-06,-84.607387,35.0,59,1.0,0.672222,0.777778,0.666667,0.510303,0.409524
79942,60110,99967,2016-09-11,2016-09-13,363,40,ecru,40,64.90,48237,...,2021-07-06,2.000000,55.0,77,0.0,0.668651,0.444444,0.527277,0.355311,1.000000
79943,40007,99968,2016-09-11,2016-09-13,259,40,mint,1,89.90,48237,...,2021-07-06,2.000000,55.0,77,0.0,0.541025,0.666667,0.527277,0.355311,0.333333


## Getting return rates by various variables to analysed the differentiating variables

In [44]:
col = ['order_date', 'item_id', 'item_size',
       'item_color', 'brand_id', 'item_price', 'user_id', 'user_title',
       'user_state', 'Delivery_Time',
        'System_Age_months', 'User_Age_years','user_return_history','brand_id_return_trend','item_id_return_trend','item_size_return_trend']

In [45]:

with pd.ExcelWriter(path + 'Return_rates_groups_20210704v1.xlsx') as writer:
    
    for each in col:

        print(each)

        test = pd.DataFrame

        test = df.groupby([each]).agg(Total_Returns=('return', 'sum'),Total_Orders=('return', 'count'))
        test['Return_per'] = test['Total_Returns']/test['Total_Orders']
        test.to_excel(writer, sheet_name= each)

order_date
item_id
item_size
item_color
brand_id
item_price
user_id
user_title
user_state
Delivery_Time
System_Age_months
User_Age_years
user_return_history
brand_id_return_trend
item_id_return_trend
item_size_return_trend


In [46]:
#bins = [0, 20, 40,60,80, 100]
#df['binned_age'] = np.searchsorted(bins, df['User_Age_years'].values)
#df['binned_System_Age'] = np.searchsorted(bins, df['System_Age_months'].values)
#df['binned_Delivery_Time'] = np.searchsorted(bins, df['Delivery_Time'].values)
#df['binned_item_price'] = np.searchsorted(bins, df['item_price'].values)


    
    

# Section 3 - Building the models 

### Creating dummy variables for the categorical variables in the data

In [ ]:
##### Get the return type variable in the 

df_updated =pd.get_dummies(df, columns=["item_id","item_size","item_color","brand_id","user_title","user_state"])

In [48]:
df_updated ['Price'] = df_updated ['item_price'] 

In [49]:
df_updated = df_updated.dropna()
df_updated = df_updated.reset_index()
df_updated.iloc[:,11:]

,Delivery_Time,User_Age_years,System_Age_months,user_return_history,brand_id_return_trend,item_id_return_trend,item_size_return_trend,user_state_return_trend,item_color_return_trend,item_id_1,...,user_state_1008,user_state_1009,user_state_1010,user_state_1011,user_state_1012,user_state_1013,user_state_1014,user_state_1015,user_state_1016,Price
0,5.000000,52.0,61,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,...,0,0,0,0,0,1,0,0,0,49.90
1,5.000000,75.0,72,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,...,0,0,1,0,0,0,0,0,0,24.90
2,5.000000,75.0,72,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,...,0,0,1,0,0,0,0,0,0,39.90
3,13.000000,51.0,76,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,...,0,0,0,0,0,0,0,0,0,79.90
4,5.000000,51.0,76,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,...,0,0,0,0,0,0,0,0,0,19.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66157,2.000000,55.0,77,0.0,1.000000,1.000000,0.527277,0.355311,0.409524,0,...,0,0,0,0,0,0,0,0,0,85.00
66158,-84.607387,35.0,59,1.0,0.672222,0.777778,0.666667,0.510303,0.409524,0,...,0,0,0,0,0,0,0,0,0,49.95
66159,2.000000,55.0,77,0.0,0.668651,0.444444,0.527277,0.355311,1.000000,0,...,0,0,0,0,0,0,0,0,0,64.90
66160,2.000000,55.0,77,0.0,0.541025,0.666667,0.527277,0.355311,0.333333,0,...,0,0,0,0,0,0,0,0,0,89.90


#### Scaling the data for the model 

In [50]:

sc = StandardScaler()
X =  pd.DataFrame(sc.fit_transform(df_updated.iloc[:,11:]))

In [51]:
X.columns =df_updated.iloc[:,11:].columns
X

,Delivery_Time,User_Age_years,System_Age_months,user_return_history,brand_id_return_trend,item_id_return_trend,item_size_return_trend,user_state_return_trend,item_color_return_trend,item_id_1,...,user_state_1008,user_state_1009,user_state_1010,user_state_1011,user_state_1012,user_state_1013,user_state_1014,user_state_1015,user_state_1016,Price
0,-0.076701,-0.421599,-1.122896,-0.335554,-2.997945,-1.722669,-4.007651,-5.779201,-4.593846,-0.038907,...,-0.420375,-0.128941,-0.544430,-0.226403,-0.095099,5.497565,-0.104448,-0.241533,-0.128212,-0.323767
1,-0.076701,1.618129,0.381200,-0.335554,-2.997945,-1.722669,-4.007651,-5.779201,-4.593846,-0.038907,...,-0.420375,-0.128941,1.836782,-0.226403,-0.095099,-0.181899,-0.104448,-0.241533,-0.128212,-0.845079
2,-0.076701,1.618129,0.381200,-0.335554,-2.997945,-1.722669,-4.007651,-5.779201,-4.593846,-0.038907,...,-0.420375,-0.128941,1.836782,-0.226403,-0.095099,-0.181899,-0.104448,-0.241533,-0.128212,-0.532292
3,0.404797,-0.510283,0.928144,-0.335554,-2.997945,-1.722669,-4.007651,-5.779201,-4.593846,-0.038907,...,-0.420375,-0.128941,-0.544430,-0.226403,-0.095099,-0.181899,-0.104448,-0.241533,-0.128212,0.301809
4,-0.076701,-0.510283,0.928144,-0.335554,-2.997945,-1.722669,-4.007651,-5.779201,-4.593846,-0.038907,...,-0.420375,-0.128941,-0.544430,-0.226403,-0.095099,-0.181899,-0.104448,-0.241533,-0.128212,-0.949342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66157,-0.257262,-0.155547,1.064880,-0.335554,3.503164,2.144643,0.592515,-1.289860,-0.511258,-0.038907,...,-0.420375,-0.128941,-0.544430,-0.226403,-0.095099,-0.181899,-0.104448,-0.241533,-0.128212,0.408156
66158,-5.469917,-1.929224,-1.396368,-0.007342,1.372245,1.285240,1.808604,0.668452,-0.511258,-0.038907,...,-0.420375,-0.128941,-0.544430,-0.226403,-0.095099,-0.181899,-0.104448,-0.241533,-0.128212,-0.322724
66159,-0.257262,-0.155547,1.064880,-0.335554,1.349027,-0.003864,0.592515,-1.289860,5.375264,-0.038907,...,-0.420375,-0.128941,-0.544430,-0.226403,-0.095099,-0.181899,-0.104448,-0.241533,-0.128212,-0.010979
66160,-0.257262,-0.155547,1.064880,-0.335554,0.519319,0.855539,0.592515,-1.289860,-1.270809,-0.038907,...,-0.420375,-0.128941,-0.544430,-0.226403,-0.095099,-0.181899,-0.104448,-0.241533,-0.128212,0.510334


In [52]:
df_updated.iloc[:,11:]

,Delivery_Time,User_Age_years,System_Age_months,user_return_history,brand_id_return_trend,item_id_return_trend,item_size_return_trend,user_state_return_trend,item_color_return_trend,item_id_1,...,user_state_1008,user_state_1009,user_state_1010,user_state_1011,user_state_1012,user_state_1013,user_state_1014,user_state_1015,user_state_1016,Price
0,5.000000,52.0,61,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,...,0,0,0,0,0,1,0,0,0,49.90
1,5.000000,75.0,72,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,...,0,0,1,0,0,0,0,0,0,24.90
2,5.000000,75.0,72,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,...,0,0,1,0,0,0,0,0,0,39.90
3,13.000000,51.0,76,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,...,0,0,0,0,0,0,0,0,0,79.90
4,5.000000,51.0,76,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,...,0,0,0,0,0,0,0,0,0,19.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66157,2.000000,55.0,77,0.0,1.000000,1.000000,0.527277,0.355311,0.409524,0,...,0,0,0,0,0,0,0,0,0,85.00
66158,-84.607387,35.0,59,1.0,0.672222,0.777778,0.666667,0.510303,0.409524,0,...,0,0,0,0,0,0,0,0,0,49.95
66159,2.000000,55.0,77,0.0,0.668651,0.444444,0.527277,0.355311,1.000000,0,...,0,0,0,0,0,0,0,0,0,64.90
66160,2.000000,55.0,77,0.0,0.541025,0.666667,0.527277,0.355311,0.333333,0,...,0,0,0,0,0,0,0,0,0,89.90


#### Splitting the data into test and training set

In [53]:
#df_updated.columns[12:]

#df_updated = df_updated.dropna()
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(X, df_updated['return'], test_size=0.3)


### Building the random forrest models using hyper parameter optmization

In [54]:
import random
random.seed(1213) 


#name = currency_name+"_"+condition+"_"+tf

param_grid = { 
'n_estimators': [500],
'max_features': [ 'sqrt'],
'max_depth':[15],
'min_samples_leaf' :[2]  
}

rfc = RandomForestClassifier()

CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
model =CV_rfc.fit(X_train, y_train)
best_params = pd.DataFrame()
best_params = CV_rfc.best_params_


#best_params

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

best_params

In [55]:
rfc_predict = pd.DataFrame(model.predict_proba(X_test))
rfc_predict

,0,1
0,0.474248,0.525752
1,0.419104,0.580896
2,0.469496,0.530504
3,0.553389,0.446611
4,0.510314,0.489686
...,...,...
19844,0.497185,0.502815
19845,0.493184,0.506816
19846,0.490426,0.509574
19847,0.411529,0.588471


In [56]:
y_test =y_test.reset_index()

final_pred = pd.concat([rfc_predict,y_test],axis =1)
final_pred

,0,1,index,return
0,0.474248,0.525752,13618,0
1,0.419104,0.580896,28860,0
2,0.469496,0.530504,22085,0
3,0.553389,0.446611,62991,0
4,0.510314,0.489686,63544,1
...,...,...,...,...
19844,0.497185,0.502815,22024,1
19845,0.493184,0.506816,8027,0
19846,0.490426,0.509574,58102,0
19847,0.411529,0.588471,63296,1


In [57]:
final_pred.columns =['Prob_0','Prob_1','index','return']

# Creating the predcited column in the data using probablity cut off

In [134]:
final_pred['Predicted'] = np.where(final_pred['Prob_1']>=0.51,1,0)

In [135]:
final_pred['Prob_1'].describe()

count    19849.000000
mean         0.505801
std          0.055129
min          0.224198
25%          0.476273
50%          0.516638
75%          0.546013
max          0.647995
Name: Prob_1, dtype: float64

In [136]:
Accuracy_table =final_pred.groupby(['return']).agg(Predicted_mean=('Predicted', 'mean'),Predicted_1=('Predicted', 'sum'),Predicted_0=('Predicted', 'count'))
Accuracy_table= Accuracy_table.reset_index()
Accuracy_table.columns = ['Actual_return','predicted_mean','predicted_1','predicted_all']
Accuracy_table['predicted_0'] = Accuracy_table['predicted_all'] - Accuracy_table['predicted_1']
Accuracy = (Accuracy_table['predicted_0'][0] + Accuracy_table['predicted_1'][1] )/Accuracy_table['predicted_all'].sum()
Accuracy

0.5991233815305557

### Creating the confusion matrix table to asses the model perfromance and create lift chart

In [61]:

p_table =pd.DataFrame()

list_a = [10,20,30,40,50,60,70,80,90]

i=0

for n in list_a:
           

            p_table.loc[i,'Percentile'] = n

            p_table.loc[i,'Cut_Off_Probability'] = np.percentile(np.array(final_pred['Prob_1']),n)

            cut_off = p_table.loc[i,'Cut_Off_Probability']

            p_table.loc[i,'TN'] = len(final_pred[final_pred['return'] == 0]\
                                      [final_pred['Prob_1'] < cut_off])

            p_table.loc[i,'FP'] = len(final_pred[final_pred['return'] == 0]\
                                      [final_pred['Prob_1'] >= cut_off])

            p_table.loc[i,'FN'] = len(final_pred[final_pred['return'] == 1]\
                                      [final_pred['Prob_1'] < cut_off])

            p_table.loc[i,'TP'] = len(final_pred[final_pred['return'] == 1]\
                                      [final_pred['Prob_1'] >= cut_off])

            p_table.loc[i,'Total'] = p_table.loc[i,'TN'] + p_table.loc[i,'FP'] + p_table.loc[i,'FN'] + p_table.loc[i,'TP']

            p_table.loc[i,'FP+TP'] = p_table.loc[i,'FP'] + p_table.loc[i,'TP']

        

            p_table.loc[i,'Predicted 1 Accuracy'] = p_table.loc[i,'TP']/(p_table.loc[i,'TP'] + p_table.loc[i,'FP'])

          

            i+=1



p_table.to_csv("Lift_chart_RF_0706V1.csv")


C:\Users\Avinash Mishra\Avinash\anaconda20\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\Avinash Mishra\Avinash\anaconda20\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\Avinash Mishra\Avinash\anaconda20\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\Avinash Mishra\Avinash\anaconda20\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


#### Getting the relative importance of variables in the models 

In [62]:
from sklearn.metrics import r2_score
from rfpimp import permutation_importances

def r2(model, X_train, y_train):
    return r2_score(y_train, model.predict(X_train))

perm_imp_rfpimp = permutation_importances(model, X_train, y_train, r2)

C:\Users\Avinash Mishra\Avinash\anaconda20\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [64]:
perm_imp_rfpimp.to_csv('relative_importance_v1.csv')

###  Building the Neural Network models 

In [35]:
import random
random.seed(1) 


import tensorflow
tensorflow.random.set_seed(3)

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from IPython.display import clear_output
from numpy import arange
import datetime as dt
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
import os
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix


from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix


from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from keras import Sequential
from keras.layers import Dense
import pickle
from sklearn.preprocessing import StandardScaler

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV


from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid
import random 
import time
from keras.layers import LeakyReLU
import h5py


from keras import Sequential
from keras.layers import Conv2D, Flatten, Dense
#import shap

#### creating a 3 hidden layer NN models with data

In [36]:
cnn = Sequential()
#--------------First Hidden Layer----------------------------#
cnn.add(Dense(units = 8, activation='relu', kernel_initializer='he_uniform',input_dim= len(X_train.columns)))

#--------------Second  Hidden Layer -------------------------#
cnn.add(Dense(4, activation='relu', kernel_initializer='he_uniform'))

#--------------Third Hidden Layer -------------------------#
cnn.add(Dense(2, activation='relu', kernel_initializer='he_uniform'))

#--------------Output Layer----------------------------------#
cnn.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

#--------------Compiling the neural network------------------#
cnn.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

model = cnn.fit(X_train,y_train, batch_size=100, epochs=1000)

Epoch 1/1000
464/464 [==============================] - 2s 4ms/step - loss: 0.6844 - accuracy: 0.5585
Epoch 2/1000
464/464 [==============================] - 2s 4ms/step - loss: 0.6673 - accuracy: 0.6008
Epoch 3/1000
464/464 [==============================] - 2s 5ms/step - loss: 0.6596 - accuracy: 0.6083
Epoch 4/1000
464/464 [==============================] - 2s 4ms/step - loss: 0.6541 - accuracy: 0.6157
Epoch 5/1000
464/464 [==============================] - 2s 5ms/step - loss: 0.6499 - accuracy: 0.6219
Epoch 6/1000
464/464 [==============================] - 2s 4ms/step - loss: 0.6463 - accuracy: 0.6268
Epoch 7/1000
464/464 [==============================] - 2s 5ms/step - loss: 0.6423 - accuracy: 0.6327
Epoch 8/1000
464/464 [==============================] - 4s 8ms/step - loss: 0.6387 - accuracy: 0.6385
Epoch 9/1000
464/464 [==============================] - 6s 12ms/step - loss: 0.6353 - accuracy: 0.6413 1s - loss: 0.6341 - accuracy
Epoch 10/1000
464/464 [=============================

464/464 [==============================] - 2s 4ms/step - loss: 0.5481 - accuracy: 0.7279
Epoch 159/1000
464/464 [==============================] - 2s 5ms/step - loss: 0.5478 - accuracy: 0.7280
Epoch 160/1000
464/464 [==============================] - 2s 5ms/step - loss: 0.5480 - accuracy: 0.7278
Epoch 161/1000
464/464 [==============================] - 2s 4ms/step - loss: 0.5475 - accuracy: 0.7283
Epoch 162/1000
464/464 [==============================] - 2s 4ms/step - loss: 0.5473 - accuracy: 0.7290
Epoch 163/1000
464/464 [==============================] - 2s 5ms/step - loss: 0.5470 - accuracy: 0.7283
Epoch 164/1000
464/464 [==============================] - 2s 5ms/step - loss: 0.5474 - accuracy: 0.7285
Epoch 165/1000
464/464 [==============================] - 2s 4ms/step - loss: 0.5474 - accuracy: 0.7285
Epoch 166/1000
464/464 [==============================] - 2s 4ms/step - loss: 0.5474 - accuracy: 0.7282
Epoch 167/1000
464/464 [==============================] - 2s 4ms/step - loss: 0

464/464 [==============================] - 1s 3ms/step - loss: 0.5375 - accuracy: 0.7358
Epoch 315/1000
464/464 [==============================] - 1s 3ms/step - loss: 0.5369 - accuracy: 0.7358
Epoch 316/1000
464/464 [==============================] - 2s 3ms/step - loss: 0.5364 - accuracy: 0.7355
Epoch 317/1000
464/464 [==============================] - 2s 3ms/step - loss: 0.5370 - accuracy: 0.7361
Epoch 318/1000
464/464 [==============================] - 2s 3ms/step - loss: 0.5373 - accuracy: 0.7358
Epoch 319/1000
464/464 [==============================] - 2s 3ms/step - loss: 0.5377 - accuracy: 0.7355
Epoch 320/1000
464/464 [==============================] - 2s 4ms/step - loss: 0.5375 - accuracy: 0.7366
Epoch 321/1000
464/464 [==============================] - 2s 3ms/step - loss: 0.5372 - accuracy: 0.7356
Epoch 322/1000
464/464 [==============================] - 2s 4ms/step - loss: 0.5383 - accuracy: 0.7353
Epoch 323/1000
464/464 [==============================] - 2s 4ms/step - loss: 0

464/464 [==============================] - 4s 8ms/step - loss: 0.5341 - accuracy: 0.7398
Epoch 471/1000
464/464 [==============================] - 3s 6ms/step - loss: 0.5329 - accuracy: 0.7396
Epoch 472/1000
464/464 [==============================] - 4s 9ms/step - loss: 0.5334 - accuracy: 0.7394
Epoch 473/1000
464/464 [==============================] - 4s 9ms/step - loss: 0.5334 - accuracy: 0.7391
Epoch 474/1000
464/464 [==============================] - 4s 8ms/step - loss: 0.5334 - accuracy: 0.7399
Epoch 475/1000
464/464 [==============================] - 3s 8ms/step - loss: 0.5338 - accuracy: 0.7393
Epoch 476/1000
464/464 [==============================] - 3s 6ms/step - loss: 0.5336 - accuracy: 0.7392
Epoch 477/1000
464/464 [==============================] - 3s 7ms/step - loss: 0.5339 - accuracy: 0.7388
Epoch 478/1000
464/464 [==============================] - 3s 7ms/step - loss: 0.5326 - accuracy: 0.7406
Epoch 479/1000
464/464 [==============================] - 3s 6ms/step - loss: 0

464/464 [==============================] - 2s 4ms/step - loss: 0.5300 - accuracy: 0.7420
Epoch 626/1000
464/464 [==============================] - ETA: 0s - loss: 0.5303 - accuracy: 0.74 - 2s 4ms/step - loss: 0.5304 - accuracy: 0.7419
Epoch 627/1000
464/464 [==============================] - 2s 4ms/step - loss: 0.5306 - accuracy: 0.7421
Epoch 628/1000
464/464 [==============================] - 2s 4ms/step - loss: 0.5304 - accuracy: 0.7417
Epoch 629/1000
464/464 [==============================] - 2s 4ms/step - loss: 0.5303 - accuracy: 0.7415
Epoch 630/1000
464/464 [==============================] - 2s 4ms/step - loss: 0.5304 - accuracy: 0.7415
Epoch 631/1000
464/464 [==============================] - 2s 4ms/step - loss: 0.5311 - accuracy: 0.7411
Epoch 632/1000
464/464 [==============================] - 2s 4ms/step - loss: 0.5298 - accuracy: 0.7418
Epoch 633/1000
464/464 [==============================] - 2s 4ms/step - loss: 0.5304 - accuracy: 0.7416
Epoch 634/1000
464/464 [=============

464/464 [==============================] - 2s 5ms/step - loss: 0.5293 - accuracy: 0.7417
Epoch 704/1000
464/464 [==============================] - 2s 5ms/step - loss: 0.5297 - accuracy: 0.7420
Epoch 705/1000
464/464 [==============================] - 2s 4ms/step - loss: 0.5297 - accuracy: 0.7422
Epoch 706/1000
464/464 [==============================] - 2s 5ms/step - loss: 0.5298 - accuracy: 0.7418
Epoch 707/1000
464/464 [==============================] - 2s 5ms/step - loss: 0.5292 - accuracy: 0.7421
Epoch 708/1000
464/464 [==============================] - 2s 5ms/step - loss: 0.5304 - accuracy: 0.7416
Epoch 709/1000
464/464 [==============================] - 2s 4ms/step - loss: 0.5301 - accuracy: 0.7421
Epoch 710/1000
464/464 [==============================] - 3s 6ms/step - loss: 0.5300 - accuracy: 0.7431
Epoch 711/1000
464/464 [==============================] - 3s 7ms/step - loss: 0.5291 - accuracy: 0.7429: 0s - loss: 0.5296 - ac
Epoch 712/1000
464/464 [==============================]

464/464 [==============================] - 2s 4ms/step - loss: 0.5278 - accuracy: 0.7442
Epoch 859/1000
464/464 [==============================] - 3s 7ms/step - loss: 0.5279 - accuracy: 0.7450
Epoch 860/1000
464/464 [==============================] - 2s 5ms/step - loss: 0.5279 - accuracy: 0.7434
Epoch 861/1000
464/464 [==============================] - 2s 4ms/step - loss: 0.5283 - accuracy: 0.7427
Epoch 862/1000
464/464 [==============================] - 3s 7ms/step - loss: 0.5277 - accuracy: 0.7440
Epoch 863/1000
464/464 [==============================] - 2s 4ms/step - loss: 0.5281 - accuracy: 0.7441
Epoch 864/1000
464/464 [==============================] - 2s 4ms/step - loss: 0.5289 - accuracy: 0.7420
Epoch 865/1000
464/464 [==============================] - 2s 4ms/step - loss: 0.5286 - accuracy: 0.7439
Epoch 866/1000
464/464 [==============================] - 2s 4ms/step - loss: 0.5284 - accuracy: 0.7431
Epoch 867/1000
464/464 [==============================] - 2s 4ms/step - loss: 0

In [40]:
cnn_predict = pd.DataFrame(cnn.predict(X_test))
cnn_predict

,0
0,0.703418
1,0.703418
2,0.703418
3,0.322655
4,0.007281
...,...
19844,0.703418
19845,0.703418
19846,0.380799
19847,0.703418


In [38]:
y_test = y_test.reset_index()

In [43]:
final_pred_cnn = pd.concat([cnn_predict,y_test],axis =1)

final_pred_cnn.columns =['Prob_1','level_0','index','return']
final_pred_cnn

,Prob_1,level_0,index,return
0,0.703418,0,23110,1
1,0.703418,1,44220,1
2,0.703418,2,30942,1
3,0.322655,3,33417,0
4,0.007281,4,43562,1
...,...,...,...,...
19844,0.703418,19844,61162,0
19845,0.703418,19845,41122,1
19846,0.380799,19846,16689,1
19847,0.703418,19847,18778,0


## Creating the predcited column in the data using probablity cut off

In [52]:
#final_pred.columns =['Prob_1','index','return']
final_pred_cnn['Predicted_cnn'] = np.where(final_pred_cnn['Prob_1']>=0.65,1,0)
final_pred_cnn.groupby(['return']).agg(Predicted_mean=('Predicted_cnn', 'mean'),Predicted_1=('Predicted_cnn', 'sum'),Predicted_all=('Predicted_cnn', 'count'))

,Predicted_mean,Predicted_1,Predicted_all
return,,,
0,0.550241,5476,9952
1,0.690714,6836,9897


In [53]:
Accuracy_table =final_pred_cnn.groupby(['return']).agg(Predicted_mean=('Predicted_cnn', 'mean'),Predicted_1=('Predicted_cnn', 'sum'),Predicted_0=('Predicted_cnn', 'count'))
Accuracy_table= Accuracy_table.reset_index()
Accuracy_table.columns = ['Actual_return','predicted_mean','predicted_1','predicted_all']
Accuracy_table['predicted_0'] = Accuracy_table['predicted_all'] - Accuracy_table['predicted_1']
Accuracy = (Accuracy_table['predicted_0'][0] + Accuracy_table['predicted_1'][1] )/Accuracy_table['predicted_all'].sum()
Accuracy

0.5699027658824122

In [54]:
final_pred_cnn

,Prob_1,level_0,index,return,Predicted_cnn
0,0.703418,0,23110,1,1
1,0.703418,1,44220,1,1
2,0.703418,2,30942,1,1
3,0.322655,3,33417,0,0
4,0.007281,4,43562,1,0
...,...,...,...,...,...
19844,0.703418,19844,61162,0,1
19845,0.703418,19845,41122,1,1
19846,0.380799,19846,16689,1,0
19847,0.703418,19847,18778,0,1


## Creating the confusion matrix table to asses the model perfromance and create lift chart

In [55]:

p_table =pd.DataFrame()

list_a = [10,20,30,40,50,60,70,80,90]

i=0

for n in list_a:
           

            p_table.loc[i,'Percentile'] = n

            p_table.loc[i,'Cut_Off_Probability'] = np.percentile(np.array(final_pred_cnn['Prob_1']),n)

            cut_off = p_table.loc[i,'Cut_Off_Probability']

            p_table.loc[i,'TN'] = len(final_pred_cnn[final_pred_cnn['return'] == 0]\
                                      [final_pred_cnn['Prob_1'] < cut_off])

            p_table.loc[i,'FP'] = len(final_pred_cnn[final_pred_cnn['return'] == 0]\
                                      [final_pred_cnn['Prob_1'] >= cut_off])

            p_table.loc[i,'FN'] = len(final_pred_cnn[final_pred_cnn['return'] == 1]\
                                      [final_pred_cnn['Prob_1'] < cut_off])

            p_table.loc[i,'TP'] = len(final_pred_cnn[final_pred_cnn['return'] == 1]\
                                      [final_pred_cnn['Prob_1'] >= cut_off])

            p_table.loc[i,'Total'] = p_table.loc[i,'TN'] + p_table.loc[i,'FP'] + p_table.loc[i,'FN'] + p_table.loc[i,'TP']

            p_table.loc[i,'FP+TP'] = p_table.loc[i,'FP'] + p_table.loc[i,'TP']

        

            p_table.loc[i,'Predicted 1 Accuracy'] = p_table.loc[i,'TP']/(p_table.loc[i,'TP'] + p_table.loc[i,'FP'])

          

            i+=1



p_table.to_csv("Cnn_lift_chart.csv")


C:\Users\Avinash Mishra\Avinash\anaconda20\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\Avinash Mishra\Avinash\anaconda20\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\Avinash Mishra\Avinash\anaconda20\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\Avinash Mishra\Avinash\anaconda20\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


#### NN Models are not perfroming and will need to be iterated further for improvments

# Section 4  - Prediction of the Validation data 

In [88]:
#### create some derived varibale in the data 

df_validation = pd.read_csv(path + "TestingData_For_Candidate.csv")

df_validation['delivery_date']=pd.to_datetime(df_validation['delivery_date'],format = '%m/%d/%Y')
df_validation['order_date']=pd.to_datetime(df_validation['order_date'],format = '%m/%d/%Y')
df_validation['user_reg_date']=pd.to_datetime(df_validation['user_reg_date'],format = '%m/%d/%Y')
df_validation['user_dob']=pd.to_datetime(df_validation['user_dob'],format = '%m/%d/%Y')
df_validation['Today'] =pd.to_datetime(date.today(),format = '%Y-%m-%d')


df_validation['Delivery_Time'] = (df_validation['delivery_date'] -df_validation['order_date'])// timedelta(days=1)
df_validation['User_Age_years'] = (df_validation['Today'] - df_validation['user_dob'])// timedelta(days=365.2425)
df_validation['System_Age_months'] =(df_validation['Today'] - df_validation['user_reg_date'])// timedelta(days=30)

In [89]:
df_validation['Delivery_Time'] = np.where(df_validation['Delivery_Time']< 0, df_validation['Delivery_Time'].mean(),df_validation['Delivery_Time'] )

In [90]:
df_validation['Delivery_Time'] = df_validation['Delivery_Time'].fillna(df_validation['Delivery_Time'].mean())
df_validation['Delivery_Time']

0         3.000000
1         9.000000
2         3.000000
3         6.269396
4         6.269396
           ...    
20050     2.000000
20051    66.000000
20052     3.000000
20053     1.000000
20054     1.000000
Name: Delivery_Time, Length: 20055, dtype: float64

##### Addition of variables using the historical return rates to the validation dataset

In [91]:
##### Addtiion of User Return History

test = df[['user_return_history','order_date','user_id']]
test.drop_duplicates(keep = False, inplace = True)
test2 =  df_validation[['order_date','user_id']]
test2.drop_duplicates(keep = False, inplace = True)
test = test.append(test2)
test = test.sort_values(by=['user_id','order_date'])
test.drop_duplicates(keep = False, inplace = True)
test['user_return_history'] = test['user_return_history'].ffill()
test =test.drop_duplicates(subset=['order_date', 'user_id'])
df_validation = df_validation.merge(test, how ='left', on=['user_id','order_date'])
df_validation['user_return_history']= df_validation['user_return_history'].fillna(0)


C:\Users\Avinash Mishra\Avinash\anaconda20\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Avinash Mishra\Avinash\anaconda20\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [92]:
df_validation

,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,Today,Delivery_Time,User_Age_years,System_Age_months,user_return_history
0,26,2016-06-23,2016-06-26,92,xl,turquoise,42,69.90,9392,Mrs,1962-04-14,1010,2016-03-24,2021-07-06,3.000000,59.0,64,0.0
1,28,2016-06-23,2016-07-02,2,xxl,green,2,19.90,9392,Mrs,1962-04-14,1010,2016-03-24,2021-07-06,9.000000,59.0,64,0.0
2,37,2016-06-23,2016-06-26,895,38,white,39,39.95,30826,Mrs,1964-04-27,1001,2015-02-17,2021-07-06,3.000000,57.0,77,0.0
3,56,2016-06-23,NaT,5,l,white,5,69.90,30828,Mrs,1966-05-13,1008,2016-01-21,2021-07-06,6.269396,55.0,66,0.0
4,65,2016-06-23,NaT,55,40,purple,1,89.90,22948,Mrs,1957-03-11,1001,2015-02-17,2021-07-06,6.269396,64.0,77,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20050,99935,2016-09-11,2016-09-13,2217,128,red,42,79.90,48229,Mrs,1965-11-15,1001,2015-10-06,2021-07-06,2.000000,55.0,70,0.0
20051,99943,2016-09-11,2016-11-16,1652,39,purple,47,39.90,48233,Mrs,1959-06-30,1008,2016-09-12,2021-07-06,66.000000,62.0,58,0.0
20052,99952,2016-09-11,2016-09-14,1611,40,black,70,249.90,48234,Mrs,1962-10-02,1007,2016-09-12,2021-07-06,3.000000,58.0,58,0.0
20053,99986,2016-09-11,2016-09-12,1550,l,berry,117,129.90,12130,Mrs,NaT,1008,2016-01-20,2021-07-06,1.000000,NaN,66,0.0


In [93]:
df_validation['user_return_history'].describe()

count    20055.000000
mean         0.421840
std          1.606764
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         48.000000
Name: user_return_history, dtype: float64

#### adding the other derived trend variables using the traning data 

In [94]:
variable_list = ['brand_id','item_id','item_size','user_state','item_color']

for each in variable_list :
    
    var = str(each) + "_return_trend"
    
    test = df[['order_date',var,each]]
    
    
    test.drop_duplicates(keep = False, inplace = True)
    
    test2 =  df_validation[['order_date',each]]
    
    test2.drop_duplicates(keep = False, inplace = True)
    
    
    test = test.append(test2)
    
    test = test.sort_values(by=[each,'order_date'])
    
    
    test.drop_duplicates(keep = False, inplace = True)
    
    test[var] = test[var].ffill()
    
    test =test.drop_duplicates(subset=['order_date', each])
    
    
    df_validation = df_validation.merge(test, how ='left', on=[each,'order_date'])
    df_validation[var]= df_validation[var].fillna(0)


df_validation ['Price'] = df_validation ['item_price'] 

C:\Users\Avinash Mishra\Avinash\anaconda20\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Avinash Mishra\Avinash\anaconda20\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### ADDING THE DUMMY VARIABLES

In [95]:
df_validation_updated =pd.get_dummies(df_validation, columns=["item_id","item_size","item_color","brand_id","user_title","user_state"])

In [96]:
df_validation_updated

,order_item_id,order_date,delivery_date,item_price,user_id,user_dob,user_reg_date,Today,Delivery_Time,User_Age_years,...,user_state_1007,user_state_1008,user_state_1009,user_state_1010,user_state_1011,user_state_1012,user_state_1013,user_state_1014,user_state_1015,user_state_1016
0,26,2016-06-23,2016-06-26,69.90,9392,1962-04-14,2016-03-24,2021-07-06,3.000000,59.0,...,0,0,0,1,0,0,0,0,0,0
1,28,2016-06-23,2016-07-02,19.90,9392,1962-04-14,2016-03-24,2021-07-06,9.000000,59.0,...,0,0,0,1,0,0,0,0,0,0
2,37,2016-06-23,2016-06-26,39.95,30826,1964-04-27,2015-02-17,2021-07-06,3.000000,57.0,...,0,0,0,0,0,0,0,0,0,0
3,56,2016-06-23,NaT,69.90,30828,1966-05-13,2016-01-21,2021-07-06,6.269396,55.0,...,0,1,0,0,0,0,0,0,0,0
4,65,2016-06-23,NaT,89.90,22948,1957-03-11,2015-02-17,2021-07-06,6.269396,64.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20050,99935,2016-09-11,2016-09-13,79.90,48229,1965-11-15,2015-10-06,2021-07-06,2.000000,55.0,...,0,0,0,0,0,0,0,0,0,0
20051,99943,2016-09-11,2016-11-16,39.90,48233,1959-06-30,2016-09-12,2021-07-06,66.000000,62.0,...,0,1,0,0,0,0,0,0,0,0
20052,99952,2016-09-11,2016-09-14,249.90,48234,1962-10-02,2016-09-12,2021-07-06,3.000000,58.0,...,1,0,0,0,0,0,0,0,0,0
20053,99986,2016-09-11,2016-09-12,129.90,12130,NaT,2016-01-20,2021-07-06,1.000000,NaN,...,0,1,0,0,0,0,0,0,0,0


In [97]:
df_validation['user_return_history']

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
20050    0.0
20051    0.0
20052    0.0
20053    0.0
20054    0.0
Name: user_return_history, Length: 20055, dtype: float64

In [98]:
df_validation_updated['type'] = 0
df_updated['type'] = 1
df_overall= df_updated.append(df_validation_updated)

In [99]:
df_overall = df_overall.fillna(0)

In [103]:
df_overall.columns[11:]

Index(['Delivery_Time', 'User_Age_years', 'System_Age_months',
       'user_return_history', 'brand_id_return_trend', 'item_id_return_trend',
       'item_size_return_trend', 'user_state_return_trend',
       'item_color_return_trend', 'item_id_1',
       ...
       'item_id_1433', 'item_id_1449', 'item_id_1454', 'item_id_1829',
       'item_id_2176', 'item_id_2189', 'item_id_2210', 'item_id_2222',
       'item_id_2226', 'brand_id_109'],
      dtype='object', length=2289)

#### Using the overall data for standarization

In [104]:
sc = StandardScaler()
X_validation =  pd.DataFrame(sc.fit_transform(df_overall.iloc[:,11:]))

In [107]:
X_validation.columns =df_overall.iloc[:,11:].columns

In [115]:
X_validation = X_validation[X_validation['type']<0]

In [121]:
X_validation = X_validation.reset_index()

In [125]:
X_validation.columns

Index(['index', 'Delivery_Time', 'User_Age_years', 'System_Age_months',
       'user_return_history', 'brand_id_return_trend', 'item_id_return_trend',
       'item_size_return_trend', 'user_state_return_trend',
       'item_color_return_trend',
       ...
       'item_id_1433', 'item_id_1449', 'item_id_1454', 'item_id_1829',
       'item_id_2176', 'item_id_2189', 'item_id_2210', 'item_id_2222',
       'item_id_2226', 'brand_id_109'],
      dtype='object', length=2290)

In [123]:
X.columns

Index(['Delivery_Time', 'User_Age_years', 'System_Age_months',
       'user_return_history', 'brand_id_return_trend', 'item_id_return_trend',
       'item_size_return_trend', 'user_state_return_trend',
       'item_color_return_trend', 'item_id_1',
       ...
       'user_state_1008', 'user_state_1009', 'user_state_1010',
       'user_state_1011', 'user_state_1012', 'user_state_1013',
       'user_state_1014', 'user_state_1015', 'user_state_1016', 'Price'],
      dtype='object', length=2246)

In [126]:
X_validation=X_validation[X.columns]
X_validation

,Delivery_Time,User_Age_years,System_Age_months,user_return_history,brand_id_return_trend,item_id_return_trend,item_size_return_trend,user_state_return_trend,item_color_return_trend,item_id_1,...,user_state_1008,user_state_1009,user_state_1010,user_state_1011,user_state_1012,user_state_1013,user_state_1014,user_state_1015,user_state_1016,Price
0,-0.196971,0.244953,-0.693001,-0.316282,-1.541751,-1.391217,-1.647941,-1.713599,-1.682406,-0.040902,...,-0.421944,-0.127728,1.836075,-0.225216,-0.095363,-0.18165,-0.104934,-0.24203,-0.127117,0.095002
1,0.164089,0.244953,-0.693001,-0.316282,-1.541751,1.243709,-1.647941,-1.713599,-1.682406,-0.040902,...,-0.421944,-0.127728,1.836075,-0.225216,-0.095363,-0.18165,-0.104934,-0.24203,-0.127117,-0.942197
2,-0.196971,0.099556,1.079185,-0.316282,-1.541751,-1.391217,-1.647941,-1.713599,-1.682406,-0.040902,...,-0.421944,-0.127728,-0.544640,-0.225216,-0.095363,-0.18165,-0.104934,-0.24203,-0.127117,-0.526280
3,-0.000230,-0.045841,-0.420357,-0.316282,-1.541751,-1.391217,-1.647941,-1.713599,-1.682406,-0.040902,...,2.369980,-0.127728,-0.544640,-0.225216,-0.095363,-0.18165,-0.104934,-0.24203,-0.127117,0.095002
4,-0.000230,0.608446,1.079185,-0.316282,-1.541751,-1.391217,-1.647941,-1.713599,-1.682406,-0.040902,...,-0.421944,-0.127728,-0.544640,-0.225216,-0.095363,-0.18165,-0.104934,-0.24203,-0.127117,0.509882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20050,-0.257147,-0.045841,0.124931,-0.316282,-1.541751,-0.220139,-0.112835,-1.713599,-1.682406,-0.040902,...,-0.421944,-0.127728,-0.544640,-0.225216,-0.095363,-0.18165,-0.104934,-0.24203,-0.127117,0.302442
20051,3.594154,0.463049,-1.510932,-0.316282,-1.541751,0.010528,-1.647941,-1.713599,-1.682406,-0.040902,...,2.369980,-0.127728,-0.544640,-0.225216,-0.095363,-0.18165,-0.104934,-0.24203,-0.127117,-0.527318
20052,-0.196971,0.172255,-1.510932,-0.316282,-1.541751,0.950939,-1.647941,-1.713599,-1.682406,-0.040902,...,-0.421944,-0.127728,-0.544640,-0.225216,-0.095363,-0.18165,-0.104934,-0.24203,-0.127117,3.828920
20053,-0.317324,-4.044258,-0.420357,-0.316282,-1.541751,-1.391217,-1.647941,-1.713599,-1.682406,-0.040902,...,2.369980,-0.127728,-0.544640,-0.225216,-0.095363,-0.18165,-0.104934,-0.24203,-0.127117,1.339642


# prediction of the valdiation set using the Random Forrest Models

In [127]:
rfc_predict = pd.DataFrame(model.predict_proba(X_validation))
rfc_predict

,0,1
0,0.546772,0.453228
1,0.568091,0.431909
2,0.498241,0.501759
3,0.531982,0.468018
4,0.535572,0.464428
...,...,...
20050,0.532646,0.467354
20051,0.562396,0.437604
20052,0.519759,0.480241
20053,0.511933,0.488067


In [156]:

rfc_predict.columns =['Prob_0','Prob_1']

final_pred_validation =pd.DataFrame()


final_pred_validation['Predicted'] = np.where(rfc_predict['Prob_1']>=0.48,1,0)

In [157]:
final_pred_validation['Predicted'].sum()

5902

In [158]:
final_pred_validation = pd.concat([final_pred_validation,df_validation],axis =1)
final_pred_validation

,Predicted,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,...,Delivery_Time,User_Age_years,System_Age_months,user_return_history,brand_id_return_trend,item_id_return_trend,item_size_return_trend,user_state_return_trend,item_color_return_trend,Price
0,0,26,2016-06-23,2016-06-26,92,xl,turquoise,42,69.90,9392,...,3.000000,59.0,64,0.0,0.000000,0.000000,0.000000,0.0,0.0,69.90
1,0,28,2016-06-23,2016-07-02,2,xxl,green,2,19.90,9392,...,9.000000,59.0,64,0.0,0.000000,0.750000,0.000000,0.0,0.0,19.90
2,1,37,2016-06-23,2016-06-26,895,38,white,39,39.95,30826,...,3.000000,57.0,77,0.0,0.000000,0.000000,0.000000,0.0,0.0,39.95
3,0,56,2016-06-23,NaT,5,l,white,5,69.90,30828,...,6.269396,55.0,66,0.0,0.000000,0.000000,0.000000,0.0,0.0,69.90
4,0,65,2016-06-23,NaT,55,40,purple,1,89.90,22948,...,6.269396,64.0,77,0.0,0.000000,0.000000,0.000000,0.0,0.0,89.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20050,0,99935,2016-09-11,2016-09-13,2217,128,red,42,79.90,48229,...,2.000000,55.0,70,0.0,0.000000,0.333333,0.333333,0.0,0.0,79.90
20051,0,99943,2016-09-11,2016-11-16,1652,39,purple,47,39.90,48233,...,66.000000,62.0,58,0.0,0.000000,0.398990,0.000000,0.0,0.0,39.90
20052,1,99952,2016-09-11,2016-09-14,1611,40,black,70,249.90,48234,...,3.000000,58.0,58,0.0,0.000000,0.666667,0.000000,0.0,0.0,249.90
20053,1,99986,2016-09-11,2016-09-12,1550,l,berry,117,129.90,12130,...,1.000000,NaN,66,0.0,0.000000,0.000000,0.000000,0.0,0.0,129.90


In [159]:
final_pred_validation.to_csv("Final_prediction.csv")